In [ ]:
#top-left: x=335, y=287 bottom-right:x=1023, y=390

In [1]:
import cv2
import numpy as np
import mss
import time
from ultralytics import YOLO
# Load trained model
model = YOLO("best(1).pt")
model.to("cpu") 
import neat
import pickle

In [3]:
#finding the game coordinates
'''import pyautogui
import time
time.sleep(3)
while True:
    print(pyautogui.position())
    time.sleep(1)'''

'import pyautogui\nimport time\ntime.sleep(3)\nwhile True:\n    print(pyautogui.position())\n    time.sleep(1)'

In [7]:
#top-left: x-337, y-266 bottom-right: x=1027, y=410
GAME_REGION = {
    "top": 266,
    "left": 337,
    "width": 690,
    "height": 144
}

In [ ]:
#testing the trained yolo model

'''cv2.namedWindow("Dino YOLO", cv2.WINDOW_NORMAL)

with mss.mss() as sct:
    while True:
        start = time.time()

        screenshot = sct.grab(GAME_REGION)
        frame = np.array(screenshot, dtype=np.uint8)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)

        # YOLO inference
        results = model.predict(
            source=frame,
            imgsz=640,
            conf=0.2,
            verbose=False
        )

        annotated = frame.copy()

        r = results[0]
        if r.boxes is not None:
            annotated = r.plot()

        cv2.imshow("Dino YOLO", annotated)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break


cv2.destroyAllWindows()'''

In [5]:
from matplotlib import pyplot as plt
import numpy as np
from pynput.keyboard import Controller,Key
keyboard=Controller()


In [ ]:
results = model('gameover1.png')

In [ ]:
%matplotlib inline
plt.imshow(np.squeeze(results[0].plot()))
plt.show()
print(results[0].names)

In [8]:
env_y = GAME_REGION["height"]
print(env_y)

144


In [9]:
def detect_objects(frame):
    detections =model(frame,conf=0.2,verbose=False)
    objects=[]
    r=detections[0]
    if r.boxes is None:
        return objects
    #print('boxes:',r.boxes.xyxy)
    for i,box in enumerate(r.boxes.xyxy):
        x1,y1,x2,y2 = box.tolist()
        #print("detections:",box.tolist())
        cls =int(r.boxes.cls[i])
        objects.append({
            "cls":cls,
            "x1":x1,"y1":y1,
            "x2":x2,"y2":y2,
            "w":x2-x1,
            "h":y2-y1
        })
    return objects
#remove the obstacle from obstacle list if its x2 <=rex[x1] and  its not gameover --> it means the dino avoided it

In [10]:
#speed and time to collision tracker
#pixel per frame
#dx → “How fast is the obstacle coming at me per frame?”(pixel per frame)
#eps --safety floor to avoid division by zero
#current_distance → “How far away is it right now?”

#TTC → “How many frames until impact if nothing changes?”
class FTCTracker:
    def __init__(self, eps=1.0, safe_ftc=2.0):
        self.prev_dist = None
        self.eps = eps            # minimum pixels/frame
        self.safe_ftc = safe_ftc
        self.speed = 0.0          # pixels per frame

    def update(self, dist):
        """
        dist: current horizontal distance (pixels)
        """
        print("prev_dist:",self.prev_dist)
        print("speed:",self.speed)
        if self.prev_dist is not None:
            dx = self.prev_dist - dist   # pixels per frame

            if dx > self.eps:
                self.speed = dx
            else:
                self.speed = 0.0  # not approaching / noisy frame

        self.prev_dist = dist
        print("prev_dist:",self.prev_dist)

    def compute_ftc(self, dist):
        """
        Returns TTC in frames
        """
        if self.speed <= self.eps:
            return self.safe_ftc

        ftc = dist / self.speed  #frames to collisiom
        return max(ftc, 0.0)

In [11]:
GAME_HEIGHT = float(GAME_REGION["height"])
GAME_WIDTH  = float(GAME_REGION["width"])

def nomalize(kind, value):
    if kind == "height":
        return max(0.0, min(value / GAME_HEIGHT, 1.0))
    elif kind == "width":
        return max(0.0, min(value / GAME_WIDTH, 1.0))
    else:
        raise ValueError(f"Unknown normalize type: {kind}")

In [12]:
def extract_obstacles(objects,ftc_tracker):
    #nomalized horizontal distance
    #obstacle width
    #frames_to_collision
    #vertical_gap_between the ground and the bird       
    rex = None
    obstacles=[]
    for obj in objects:
        if obj["cls"]==3:
            rex=obj
        elif obj["cls"]==2:
            pass #gameover
        else:
            obstacles.append(obj)
    
        
    if rex is None or not obstacles:
        return None
    #we now to extract obstacles that are ahead of the dino
    obstacles = [o for o in obstacles if o["x1"] > rex["x2"] ]
    #obstacle closest to the dino
    if not obstacles:
        return None
    obs = min(obstacles,key=lambda o:o['x1'])
    print('closest obstacle:',obs)
    distance_to_obstacle=obs['x1']-rex['x2']
    ftc_tracker.update(distance_to_obstacle)
    ftc=ftc_tracker.compute_ftc(distance_to_obstacle)#2
    print("ftc:",ftc)
    norm_dist =nomalize('width',distance_to_obstacle)#1
    obs_w= obs['x2']-obs['x1']
    norm_obs_w =nomalize('width',obs_w)#3
    obs_h = obs['y2'] -obs['y1']
    norm_obs_h = nomalize('height',obs_h)
    print('obs_h',norm_obs_h)
    print('norm_obs_w:',norm_obs_w)
    vertical_gap = env_y-obs['y2']
    norm_vertical_gap=nomalize('height',vertical_gap)
    print("v_height",vertical_gap)#4
    print('norm_v_gap',norm_vertical_gap)
    #return obstacles
    inputs = [norm_dist,norm_obs_w,norm_vertical_gap,ftc,norm_obs_h]
    
    print("Inputs:",inputs)
    return inputs
            

In [13]:
def apply_action(action, ducking=False):
    if action == "jump":
        if ducking:
            keyboard.release(Key.down)
            ducking = False
        keyboard.press(Key.space)
        keyboard.release(Key.space)

    elif action == "duck":
        if not ducking:
            keyboard.press(Key.down)
            ducking = True

    elif action == "none":
        if ducking:
            keyboard.release(Key.down)
            ducking = False

    return ducking


In [14]:
def choose_action(outputs):
    actions = ["jump", "duck", "none"]
    return actions[np.argmax(outputs)]

In [15]:
def game_over_detected(objects):
    return any(o["cls"] == 2 for o in objects)

In [16]:
def restart_game():
    keyboard.release(Key.down)   # safety
    keyboard.press(Key.space)
    keyboard.release(Key.space)

In [17]:
#def eval_genomes(genomes,config):
def eval_genomes(genomes,config):
    MAX_FRAMES=1500
    for genome_id,genome in genomes:
        restart_game()
        net=neat.nn.FeedForwardNetwork.create(genome,config)
        frame_count=0
        genome.fitness=0.0 #start with a fitness of 0
        ftc_tracker = FTCTracker() #pixel per frame
        ducking =False
        prev_ftc =None
        with mss.mss() as sct:
            #waiting for the game to actually start
            for _ in range(5):
                frame = np.array(sct.grab(GAME_REGION))
                frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)
                objects = detect_objects(frame)
                if not game_over_detected(objects):
                    break
                restart_game()
                time.sleep(0.08)

            while True:
                    # Capture game screen
                frame = np.array(sct.grab(GAME_REGION))
                frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)
                frame_count+=1
                    # Detect objects
                objects = detect_objects(frame)
                if game_over_detected(objects):
                    genome.fitness -= 5.0
                    break
                    
                res=extract_obstacles(objects,ftc_tracker)
                inputs=res
                if inputs:
                    
                                        
                    ftc= inputs[3]
                    
                    #output = net.activate(inputs)
                    current_ftc =ftc
                    print("prev ftc",prev_ftc)
                    print("curr ftc",current_ftc)
                    if prev_ftc is not None:
        
                    # Detect sharp jump from small FTC to large FTC
                        if prev_ftc < 0.9 and current_ftc > 1.5:
                            genome.fitness += 2.0   # reward for passing obstacle
                            
                            print('passed_obstacle')
                    prev_ftc = current_ftc
                    output = net.activate(inputs)
        
                    action = choose_action(output)
                    
                    
                    ducking = apply_action(action,ducking)
    
                    genome.fitness +=0.5
                    #if frame_count >MAX_FRAMES:
                        #break

        if ducking:
            keyboard.release(Key.down)

                

In [ ]:
def eval_genomes():

   
    ftc_tracker = FTCTracker() #pixel per frame
    ducking =False
    prev_ftc =None
    passed_this_obstacle=False
    print('gg')
    frame_count=0
    with mss.mss() as sct:
            #waiting for the game to actually start
        for _ in range(5):
            frame = np.array(sct.grab(GAME_REGION))
            frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)
            objects = detect_objects(frame)
            if not game_over_detected(objects):
                break
            restart_game()
            time.sleep(0.08)

        while True:
                    # Capture game screen
            frame = np.array(sct.grab(GAME_REGION))
            frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)
            #frame_count+=1
                    # Detect objects
            objects = detect_objects(frame)
            if game_over_detected(objects):
                break
               
                    
            res=extract_obstacles(objects,ftc_tracker)
            inputs=res   
            if inputs:   
                
                ftc= inputs[3]
                
                #output = net.activate(inputs)
                current_ftc =ftc
                print("prev ftc",prev_ftc)
                print("curr ftc",current_ftc)
                print("pto",passed_this_obstacle)
                if prev_ftc is not None:
    
                # Detect sharp jump from small FTC to large FTC
                    if prev_ftc < 0.9 and current_ftc > 1.5 and not passed_this_obstacle:
                        #genome.fitness += 3.0   # reward for passing obstacle
                        passed_this_obstacle = True
                        frame_count+=1
                        print('obstacles passed:',frame_count)
                        print('passed_obstacle')
                prev_ftc = current_ftc
                passed_this_obstacle=False
                
                
    

In [ ]:
config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    "config-neat-dino.txt"
)

pop = neat.Population(config)
pop.add_reporter(neat.StdOutReporter(True))

stats = neat.StatisticsReporter()
pop.add_reporter(stats)
pickle.dump(stats, open("stats15.pkl", "wb"))


winner = pop.run(eval_genomes, 15)

with open("dino_winner1Hnofit15.pkl", "wb") as f:
    pickle.dump(winner, f)


In [ ]:
time.sleep(3)
eval_genomes()

In [18]:
#inference mode
def run_winner():

    with open("dino_winner1Hnofit15.pkl", "rb") as f:
        winner = pickle.load(f)

    config = neat.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        "config-neat-dino.txt"
    )

    net = neat.nn.FeedForwardNetwork.create(winner, config)

    restart_game()

    ftc_tracker = FTCTracker()
    ducking = False

    with mss.mss() as sct:

        # wait for game start
        for _ in range(5):
            frame = np.array(sct.grab(GAME_REGION))
            frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)

            objects = detect_objects(frame)

            if not game_over_detected(objects):
                break

            restart_game()
            time.sleep(0.1)

        prev_ftc = None

        while True:

            frame = np.array(sct.grab(GAME_REGION))
            frame = cv2.cvtColor(frame, cv2.COLOR_BGRA2BGR)

            objects = detect_objects(frame)

            if game_over_detected(objects):
                print("Game Over")
                break

            inputs = extract_obstacles(objects, ftc_tracker)

            if inputs:

                ftc = inputs[3]

                output = net.activate(inputs)

                action = choose_action(output)

                ducking = apply_action(action, ducking)

                prev_ftc = ftc

    if ducking:
        keyboard.release(Key.down)


In [ ]:
time.sleep(3)
while True:
    run_winner()

In [ ]:
import neat
import pickle
import visualize

# Load config
config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    "config-neat-dino.txt"
)

# Load winner genome
with open("dino_winner1Hnofit10.pkl", "rb") as f:
    winner = pickle.load(f)

# OPTIONAL: label nodes for clarity
#norm_dist,norm_obs_w,norm_vertical_gap,ftc,norm_obs_h
#"jump", "duck", "none"
node_names = {
    -1: "distance",
    -2: "width",
    -3: "vertical_gap",
    -4: "ftc",
    -5: "norm_obs_h",
    0: "jump",
    1: "duck",
    2: "none"
}

# Draw network
visualize.draw_net(
    config,
    winner,
    view=True,
    node_names=node_names,
    filename="dino_winner"
)
